**Implement VGG using TF**

In [2]:
# Load data
import os
import tensorflow as tf
import numpy as np

# load weights from https://dl.dropboxusercontent.com/u/50333326/vgg16.npy
path = '/home/ariel/DL/tensorflow/tutorials/'
vgg16_npy_path = os.path.join(path, 'vgg16.npy')            
data_dict = np.load(vgg16_npy_path, encoding='latin1').item()

# print keys in dictionary - 
# here it prints all the layers names in the VGG net
name = 'conv1_1'
print 'keys in layers = ', data_dict.keys(), '\n'
# tensorflow: weights are [height, width, in_channels, out_channels]
print 'weights shape in conv1_1', data_dict[name][0].shape, '\n'
print 'bias shape in conv1_1', data_dict[name][1].shape, '\n'


keys in layers =  ['conv5_1', 'fc6', 'conv5_3', 'fc7', 'fc8', 'conv5_2', 'conv4_1', 'conv4_2', 'conv4_3', 'conv3_3', 'conv3_2', 'conv3_1', 'conv1_1', 'conv1_2', 'conv2_2', 'conv2_1'] 

weights shape in conv1_1 (3, 3, 3, 64) 

bias shape in conv1_1 (64,) 



**The Conv Block in TF**

In [3]:
def _max_pool(bottom, name):
    pool = tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                          padding='SAME', name=name)
    return pool

def _bias_reshape(bweight, num_orig, num_new):
    """ Build bias weights for filter produces with `_summary_reshape`
    """
    n_averaged_elements = num_orig//num_new
    avg_bweight = np.zeros(num_new)
    for i in range(0, num_orig, n_averaged_elements):
        start_idx = i
        end_idx = start_idx + n_averaged_elements
        avg_idx = start_idx//n_averaged_elements
        if avg_idx == num_new:
            break
        avg_bweight[avg_idx] = np.mean(bweight[start_idx:end_idx])
    return avg_bweight

# need to understand this! Only for fc8, the bias is recalculated!
def get_bias(name, num_classes=None):
    bias_wights = data_dict[name][1]
    shape = data_dict[name][1].shape
    if name == 'fc8':
        bias_wights = _bias_reshape(bias_wights, shape[0], num_classes)
        shape = [num_classes]
    init = tf.constant_initializer(value=bias_wights, dtype=tf.float32)
    return tf.get_variable(name="biases", initializer=init, shape=shape)

def get_conv_filter(name):
    with tf.variable_scope("C"):
        init = tf.constant_initializer(value=data_dict[name][0], dtype=tf.float32)
        shape = data_dict[name][0].shape
        print('Layer name: %s' % name)
        print('Layer shape: %s' % str(shape))
        var = tf.get_variable(name="filter", initializer=init, shape=shape)
#     if not tf.get_variable_scope().reuse:
#         weight_decay = tf.multiply(tf.nn.l2_loss(var), self.wd,
#                                    name='weight_loss')
#         tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES,
#                              weight_decay)
    return var

def get_weights(name):
    init = tf.constant_initializer(value=data_dict[name][0], dtype=tf.float32)
    weights = tf.get_variable(name="weights", initializer=init, shape=data_dict[name][0].shape)
    return weights

def get_biases(name):
    init = tf.constant_initializer(value=data_dict[name][1], dtype=tf.float32)
    biases = tf.get_variable(name="biases", initializer=init, shape=data_dict[name][1].shape)
    return biases

def _conv_layer(bottom, name):
    with tf.variable_scope(name):# as scope:
        # get filter weights
        weights = get_weights(name)
       
        # get filter biases
        biases = get_biases(name)

        # convolve: weights * bottom
        conv = tf.nn.conv2d(bottom, weights, [1, 1, 1, 1], padding='SAME')

        # add biases: weights * bottom + biases
        conv_biases = tf.nn.bias_add(conv, biases)

        # relu: relu(weights * bottom + biases)
        conv_biases_relu = tf.nn.relu(conv_biases)

        return conv_biases_relu

** the network **

In [4]:
# this command clean all nodes in the graph + variables
tf.reset_default_graph()

# load an image
IMAGE_SIZE = 224
img_shape = (1, IMAGE_SIZE, IMAGE_SIZE, 3) # tf format (N,H,W,C)
img = np.linspace(-0.1, 0.5, num=np.prod(img_shape)).reshape(img_shape)
tf_img = tf.Variable(img, name="img", dtype=tf.float32)

_conv1_1 = _conv_layer(tf_img, 'conv1_1')
_conv1_2 = _conv_layer(_conv1_1, 'conv1_2')
_pool1   = _max_pool(_conv1_2, 'pool1')

_conv2_1 = _conv_layer(_pool1, 'conv2_1')
_conv2_2 = _conv_layer(_conv2_1, 'conv2_2')
_pool2   = _max_pool(_conv2_2, 'pool2')

_conv3_1 = _conv_layer(_pool2, 'conv3_1')
_conv3_2 = _conv_layer(_conv3_1, 'conv3_2')
_conv3_3 = _conv_layer(_conv3_2, 'conv3_3')
_pool3   = _max_pool(_conv3_3, 'pool3')

_conv4_1 = _conv_layer(_pool3, 'conv4_1')
_conv4_2 = _conv_layer(_conv4_1, 'conv4_2')
_conv4_3 = _conv_layer(_conv4_2, 'conv4_3')
_pool4   = _max_pool(_conv4_3, 'pool4')

_conv5_1 = _conv_layer(_pool4, 'conv5_1')
_conv5_2 = _conv_layer(_conv5_1, 'conv5_2')
_conv5_3 = _conv_layer(_conv5_2, 'conv5_3')
_pool5   = _max_pool(_conv5_3, 'pool5')

sess = tf.Session()
sess.run(tf.initialize_all_variables())
res = sess.run(_pool5) 
print res.shape
print res

(1, 7, 7, 512)
[[[[ 0.04152922  0.          0.         ...,  0.          0.76243585  0.        ]
   [ 0.          0.          0.         ...,  0.          0.73776311  0.        ]
   [ 0.          0.          0.         ...,  0.          0.73881972  0.        ]
   ..., 
   [ 0.          0.          0.         ...,  0.          0.72844511  0.        ]
   [ 0.          0.          0.         ...,  0.          0.79345977  0.        ]
   [ 0.          0.          0.         ...,  0.          0.78591818  0.        ]]

  [[ 0.          0.          0.         ...,  0.          0.67036343  0.        ]
   [ 0.          0.          0.         ...,  0.          0.70792282  0.        ]
   [ 0.          0.          0.         ...,  0.          0.72337979  0.        ]
   ..., 
   [ 0.          0.          0.         ...,  0.          0.72094685  0.        ]
   [ 0.06755218  0.          0.         ...,  0.          0.66223222  0.        ]
   [ 0.07043768  0.          0.         ...,  0.          0.691